# Power

## Power pins
- AVDDH(3.3 / 2.5 V) [2 x pins]
- AVDDL (1.2 V) [2 x pins]
- DVDLL (1.2 V) [VDD] [4 x pins]
- DVDDH (3.3, 2.5 or 1.8 V) [VDD] [2 x pins]
- VSS (GND)
- LDO_O (Contains control to drive FET for 1.2 V output)
	- Leave unused, use your own 1.2 V supply
- AVDD_PL (1.2 V for PLL)

### 1.2 V power supply
- Add an extra buck converter generating 1.2 V

In [2]:
import numpy as np

def get_resistor_ratio(v_out):
	return ((v_out / 0.6) - 1)

resistor_array = np.array([20, 22, 24, 27, 30, 33, 39, 47, 51, 56, 68, 75, 82, 100, 150])
vdd_targets = np.array([1.2])
# r1_arr = [30, 75, 100]
# r2_arr = [24, 24, 22]

######### DDR3L
for idx, v_target in enumerate(vdd_targets):
	print(f"---- V = {v_target} ----")
	v_target_ratio = get_resistor_ratio(v_target)
	r2_target_arr = np.round(resistor_array / v_target_ratio, 2)
	r1_target_arr = np.round(resistor_array * v_target_ratio, 2)
	print(f"{v_target} r1/r2: {v_target_ratio}")
	print(f"R2 values {r2_target_arr}")
	print(f"R1 values {r1_target_arr}")

	# for r1, r2 in zip(r1_target_arr, r2_target_arr):
	# 	if (r1 in resistor_array):
	# 		print(f"Perfect r1 found: {r1}")
	# 	if (r2 in resistor_array):
	# 		print(f"Perfect r2 found: {r2}")

	# r1 = r1_arr[idx]
	# r2 = r2_arr[idx]
	# vout_actual = 0.6*(1+r1/r2)
	# print(f"offset: {((vout_actual - v_target) / v_target) * 100:.2f}%")

---- V = 1.2 ----
1.2 r1/r2: 1.0
R2 values [ 20.  22.  24.  27.  30.  33.  39.  47.  51.  56.  68.  75.  82. 100.
 150.]
R1 values [ 20.  22.  24.  27.  30.  33.  39.  47.  51.  56.  68.  75.  82. 100.
 150.]


**Chose 2 x 100 kOhm**

## Decoupling capacitors / Ferrites

Reference: Check page 9, ANLAN206 – KSZ9031 G

- Ferrites: 
	- < 0.1 ohms in DC Resistance
	- 60 ohm impedance at 100 MHz
	- example: MPZ1005S600C
- Capacitors
	- X7R / X5R
	- Mostly HF decoupling (Do bulk at source)

### AVDD_PLL (1.2 V)
- Ferrite
- 10 uF bulk
- 100 nF decoupling

### AVDDL (1.2)
- Ferrite
- 10 uF bulk
- 100 nF per pin


### AVDDH (3.3 V)
- Ferrite
- 10 uF X5R bulk capacitance
- 1 uF X7R bulk capacitance
- 0.1 uF decoupling per pin

### DVDDL (1.2 V)
- 100 nF decoupling on each pin
- Bulk capacitance close enough (10 uF)

### DVDDH (3.3 V)
- 100 nF per pin
- 10 uF bulk

## Notes
### Filtering with ferrites vs filtering with inductors
Ferrites
- Poorly characterized
- Dissipate resistively (also at DC!)
- Cheap
- Don't resonate

Inductors
- Well-characterized (up to a certain frequency)
	- 20 dB/dec drop, sharp resonance-filters possible here.
- Impedance peak: $f_{0}=\frac{1}{\sqrt{2*\pi*L*C}}$

# Transmit / Receive signals RGMMI
- Use Bank 1 for these signals
- We don't need any DQS groups for a RGMII interface
	- Rise/Fall/Delay time of a regular FPGA PIO:
		- LVCMOS33 has a max output frequency of 150 MHz
### Q: Ethernet speed
**How can the ethernet PHY transfer up to GB/s with 2 RX and TX data-lines, when there's 4 RX and TX data-lines to the FPGA? What are acceptable speeds on the FPGA-side?**

- Clock period for 1000BASE-T: 7.2-8.8 ns: 125 MHz
- Clock period for 100BASE-TX: 36-44 ns: 25 MHz
- Clock period for 10BASE-T: 360-440 ns: 2.5 MHz

So these are the clock and data-speeds for these signals.

Allowed skew (data sampled at clock posedge):
- -0.5 .. .5 ns for RGMII v1.3 transmitter pins
- 1.2-2.0 ns for RGMII V1.3 receiver pins

## Transmit / Receive
- TXDi
- RXDi
### RXDi latching
- RXDi, RX_CLK, RX_DV
Pull-ups, pull-downs are latched to select the relevant mode during power-on / RESET

#### Mode options
The pull-ups and pull-downs determine what capabilities are being advertised.
- On power-up it sends IEEE-802..3 auto-negotiation pulses (FLP) on the copper pairs.

Mode[3:0] - RX[3:0]
- ALL_HIGH: RGMII mode - Advertise all capabilities (10/100/1000 speed half-/full-duplex)
-> SO we need to tigh them high

#### RX_DV / CLK125_EN
- CLK125_EN
 Pull-up: enables the 125 MHz clock output on pin 41
- RX_CTL is a receive control output -> so used in receive communication.
Both RX_CTL and TX_CTL-like signals indicate validity of a frame / data transmitted.

# Clock

- XO/XI (clock input / output with crystal feedback)
- Injecting the PLL-generated signal from the PHY-chip into the FPGA
- MDIO / MDC management

### MDIO / MDC
- Management Data Clock Input
- Management data input / Output
	- Pull-up (1-4.7 kOhms to VDD_IO)
### XO/XI
CRYSTAL REQUIREMENTS:
- Choose a 25 MHz crystal
- 25 MHz +- 50 ppm tolerance
- Series resistance (40 ohms)
- Jitter (< 100 ps)
- XO: feedback, XI: input

#### Choice: X322525MOB4SI
- 50 ohm ohms Max ESR
- Shunt capacitance: 7 pF

In [4]:
import math
### Example calculation: ABLS-8.000MHZ-20-B-3-H-T
C_S = 7e-12 # Ethernet PHY datasheet
C_L = 12e-12 # Input load capacitance (resonator datasheet)
# Assume C_L1 = C_L2
C_L1 = (C_L - C_S)*2
C_L2 = C_L1
print(f"Choose load capacitances: Cl1 {C_L1*1e12}pF, Cl2 {C_L2*1e12}pF")

ESR = 50 # (resonator datasheet)
F_OSCILLATOR = 8e6
trancond = 10e-3 # in A / V, oscillator transconductance (assumed)
# Assume C_L1 = C_L2
g_m_crit = 4 * ESR * (2 * math.pi * F_OSCILLATOR) ** 2 * (C_S + C_L) **2
GM = trancond / g_m_crit
print(f"Gain margin: ({GM} > 5) ? | g_m_crit: {g_m_crit}")

Choose load capacitances: Cl1 10.000000000000002pF, Cl2 10.000000000000002pF
Gain margin: (54.81798803364017 > 5) ? | g_m_crit: 0.00018242187206621478


However: choose 22 pF anyways according to reference connection in datasheet

### PLL generated signal injection
#### CLK125_EN
Inject into
- PCLK (primary clock input for injection in primary network)
- GPLL (general-purpose PLL input pin) - Used to drive a PLL from an external clock source

Drive it into the GPLL-clock pin since we might need to derive 

#### LED MODE
- Pull-up: single-LED mode
- Pull-down: Tri-color dual-LED mode

Added LED's, white and blue, with about 3 mA going through them.

### Pin 38: INT_N / PME_N2
2 pin functions:
##### INT_N Output
- Programmable interrupt output
- When low asserted: WOL event occurred
- Requires pull-up of 1kOhm-4.7 kOhm

**Wake-On-LAN**
- MAC-based function to wake-up a host system.
- 

#### PME_N2 Output


# Transformer-Only operation


## Physical device network and signalling

- Auto-MDIX: Autonegotiation happens at 10 Mb/s and 100 Mb/s operation.
	- at 50 ohms balanced loading: differential 2.5 V signals
	- 100 Mb/s: +1 V, -1 V, 0 V operation
	- 10 Mb/s: +2.5, -2.5 V differential signals
- Transformers serve the purpose of isolating these differential voltages
	- IEEE 802.3 AC / DC isolation

## Capacitor for isolation
### Capacitor Requirements
- Return loss must be > 16 dB
- Impedance range: 100 +- 15 ohms
- Nominally resistive phase angle < 3 degrees [2, 80 MHz]

#### Acceptable capacitor value
- The acceptable value is 30.42 nF (minimum), larger will improve the impedance  and decrease the phase shift for higher frequencies.

#### Conclusion
We want to use the same DC-blocking caps for the analog front-end, as we do for the ethernet communication.
For that the uncertainty in capacitance can't lead to a phase-shift that is too big.

In order to get to that point, we will have to buy caps that are X7R, but a bit larger, OR put 2 smaller C0G caps in parallel.
- they don't form too big of an impedance at frequencies around 200 kHz, which is the minimum ADC measuring frequency
- impedance around 2 MHz, which is the IEE-requirement.



But for X7R I guess 100 kHz is still pretty ok in terms of frequency, and at higher frequency the offset is less of an issue.

## Magnetics for Isolation
Given that 
- We'd have to buy capacitors and inductors from a third party with an S.R.F that is high enough.
	- Their capacitance needs to be above 40 nF for their phase / gain uncertainty with respect to each other to be minimal.
	- We could get caps with an S.R.F. higher than 100 MHz but we'd probably have to start soldering on our own and buying them from third parties
- With a low S.R.F. the capacitance might actually start behaving as an inductance.

### LED termination
- 11->12 Green LED
- 14->13 Yellow LED (in reality unnecessary)
### Magnetics termination
- P10: 75 ohms to AC-gnd
- P1: Transformer with center tap (to be connected to Vcc / 2)

### Bob-Smith termination
- Reduces common mode currents as a consequence of coupling between different differential pairs.
- There's thus a 75 ohm resistive connection to AC ground from each line (capacitive separation)
- 

### Open-circuit inductance
Inductance measured on each phase (1-2, 3-4, ..) with all other phases open.

### Example RJ45 with integrated magnetics
- Insertion loss: -1.0 dB between 300 kHz and 100 MHz

#### RJ45 C192706 (1.14 $)
**CONSIDER USING THIS ONE FOR THE ADC INPUT**

- Only 2 differential transformers connected

![](images/rj45_100Mb.png)

This connector has 
- J7, J8 and J5, J4 connected to ground.
- 2 LED's connected

This is a typical base-10, base-100 connection
- Has a TX-pair and an RX-pair



#### RJ45 C50933 (2.47 $)
![](images/rj45_1000Mb.png)

- Typical insertion loss: 1.0 dB at 100 kHz - 60 MHz
- Return loss typically about -18 dB

### Separate magnetics
#### Balun transformer C25503618
Impedance issues:
- RJ45-connector has a 100 ohms differential impedance.
- Balun is made for single-ended 50-ohm signal transmission to 50 ohm differential signal.


Insertion loss (S21 / S12):
- 1 dB
Input return loss (S11):
- 